# Tools

A model's capabilities is extended with tools. Tools can be built into the API or accessed through functions and Model Context Protocol (MCP) services.

Tools include:

- Image generation.
- Web search.
- Function calling.
- Remote MCP Servers.
- File search.
- Code interpreter.
- Computer use.

# Function Calling

In this notebook we use function calling to illustrate tools and their usage. 

A useful resource is OpenAI's [Function Calling documentation for the responses API](https://platform.openai.com/docs/guides/function-calling?api-mode=responses).

In [2]:
%load_ext dotenv
%dotenv ../../05_src/.secrets_grassriots

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [3]:
from openai import OpenAI
import json

client = OpenAI()

# Tool Calling Flow

The idea behind using tools is that a model will request to use a tool. When a model examines a prompt, it may determine that to follow the instruction it should use a tool. It then generates a special type of output, which we will catch and call a specific tool. 

The logic flows as follows:


1. Make a request to the model with tools it could call
2. Receive a tool call from the model
3. Execute code on the application side with input from the tool call
4. Make a second request to the model with the tool output
5. Receive a final response from the model (or more tool calls)


![](./img/05_function-calling-diagram-steps.png)

# Make a request to the model with tools it could call

The tool definition includes the function name, its parameters, and additional metadata. 

In [4]:
tools = [
    {
        "type": "function",
        "name": "get_horoscope",
        "description": "Get horoscope for a given zodiac sign.",
        "parameters": {
            "type": "object",
            "properties": {
                "zodiac_sign": {
                    "type": "string",
                    "description": "Zodiac sign e.g. Aries, Taurus",
                }
            },
            "required": ["zodiac_sign"],
            "additionalProperties": False,
        },
        "strict": True,
    },
]

def get_horoscope(zodiac_sign: str) -> str:
    # Dummy implementation for illustration
    horoscope = f"{zodiac_sign}: Today is a great day for new beginnings."
    return horoscope



We will need to retain some "memory" of the conversation. In this case, we will retain the message history and send it as context in our interaction with the model

In [5]:
input_list = [
    {"role": "user", "content": "What is my horoscope? I am an Aquarius."}
]

We prompt the model with the initial input.

In [6]:
response = client.responses.create(
    model="gpt-5",
    tools=tools,
    input=input_list,
)

# Execute code on the application side with input from the tool call


Examine the response output. Notice that we find a "reasoning item" and a "function tool call".

In [13]:
response.output

[ResponseOutputMessage(id='msg_001c5b087e034f88006903e654cb88819ab1d3475d0b8a8689', content=[ResponseOutputText(annotations=[], text='Aquarius: Today is a great day for new beginnings.', type='output_text', logprobs=[])], role='assistant', status='completed', type='message')]

The function call item indicates that the model is requesting to run the function call: `get_horoscope(zodiac_sign='Aquarius')`.

In [8]:
print(response.output[1].to_json())

{
  "arguments": "{\"zodiac_sign\":\"Aquarius\"}",
  "call_id": "call_annDUn46z1PZx5Zf6WNpwZKA",
  "name": "get_horoscope",
  "type": "function_call",
  "id": "fc_001c5b087e034f88006903e5b36768819a91ec6400febba641",
  "status": "completed"
}


In the code below:

+ Execute the function logic for get_horoscope.
+ Add the function output to the input_list.

In [9]:
input_list += response.output

for item in response.output:
    if item.type == "function_call":
        if item.name == "get_horoscope":
            # Execute the function logic for get_horoscope
            horoscope = get_horoscope(**json.loads(item.arguments))
            
            # Provide function call results to the model
            input_list.append({
                "type": "function_call_output",
                "call_id": item.call_id,
                "output": json.dumps({
                  "horoscope": horoscope
                })
            })

# Make a second request to the model with the tool output


Examine the final input_list which is the context of model call.

In [10]:
print(input_list)

[{'role': 'user', 'content': 'What is my horoscope? I am an Aquarius.'}, ResponseReasoningItem(id='rs_001c5b087e034f88006903e5b1f790819ab2ee595f3e516488', summary=[], type='reasoning', content=None, encrypted_content=None, status=None), ResponseFunctionToolCall(arguments='{"zodiac_sign":"Aquarius"}', call_id='call_annDUn46z1PZx5Zf6WNpwZKA', name='get_horoscope', type='function_call', id='fc_001c5b087e034f88006903e5b36768819a91ec6400febba641', status='completed'), {'type': 'function_call_output', 'call_id': 'call_annDUn46z1PZx5Zf6WNpwZKA', 'output': '{"horoscope": "Aquarius: Today is a great day for new beginnings."}'}]


In [11]:
response = client.responses.create(
    model="gpt-5",
    instructions="Respond only with a horoscope generated by a tool.",
    tools=tools,
    input=input_list,
)


# Receive a final response from the model (or more tool calls)

In [12]:
print(response.model_dump_json(indent=2))
print("\n" + response.output_text)

{
  "id": "resp_001c5b087e034f88006903e6541c80819aa172302c61b0cc0e",
  "created_at": 1761863252.0,
  "error": null,
  "incomplete_details": null,
  "instructions": "Respond only with a horoscope generated by a tool.",
  "metadata": {},
  "model": "gpt-5-2025-08-07",
  "object": "response",
  "output": [
    {
      "id": "msg_001c5b087e034f88006903e654cb88819ab1d3475d0b8a8689",
      "content": [
        {
          "annotations": [],
          "text": "Aquarius: Today is a great day for new beginnings.",
          "type": "output_text",
          "logprobs": []
        }
      ],
      "role": "assistant",
      "status": "completed",
      "type": "message"
    }
  ],
  "parallel_tool_calls": true,
  "temperature": 1.0,
  "tool_choice": "auto",
  "tools": [
    {
      "name": "get_horoscope",
      "parameters": {
        "type": "object",
        "properties": {
          "zodiac_sign": {
            "type": "string",
            "description": "Zodiac sign e.g. Aries, Taurus"
    